<center><img src="trainers.jpg" alt="Trainers in a store" width=550></center>

Sports clothing and athleisure attire is a huge industry, worth approximately $193 billion in 2021 with a strong growth forecast over the next decade! (`Source: https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/`)

In this workbook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

### The data:
You've been provided with four datasets to investigate:

`brands.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

`finance.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

`info.csv`

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

`reviews.csv`

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [104]:
# Importing libraries
import pandas as pd

In [105]:
# Loading the data
brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In [106]:
# Taking a look into the brands dataset
brands.head()

,product_id,brand
0,AH2430,NaN
1,G27341,Adidas
2,CM0081,Adidas
3,B44832,Adidas
4,D98205,Adidas


In [107]:
# Taking a look into the finance dataset
finance.head()

,product_id,listing_price,sale_price,discount,revenue
0,AH2430,NaN,NaN,NaN,NaN
1,G27341,75.99,37.99,0.5,1641.17
2,CM0081,9.99,5.99,0.4,398.93
3,B44832,69.99,34.99,0.5,2204.37
4,D98205,79.99,39.99,0.5,5182.70


In [108]:
# Taking a look into the info dataset
info.head()

,product_name,product_id,description
0,NaN,AH2430,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor..."
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's..."
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...


In [109]:
# Taking a look into the reviews dataset
reviews.head()

,product_id,rating,reviews
0,AH2430,NaN,NaN
1,G27341,3.3,24.0
2,CM0081,2.6,37.0
3,B44832,4.1,35.0
4,D98205,3.5,72.0


In [110]:
# Answering the first questions
# Joining the brand and finance dataset
dataset_merged = pd.merge(left = brands, 
                          right = finance, 
                          on = "product_id",
                          how = "left")
dataset_merged.dropna(inplace = True)

# Creating the label column
twenty_fifth = dataset_merged["listing_price"].quantile(0.25)
median = dataset_merged["listing_price"].quantile(0.5)
seventy_fifth = dataset_merged["listing_price"].quantile(0.75)
maximum = dataset_merged["listing_price"].max()

labels = ["Budget", "Average", "Expensive", "Elite"]
bins = [0, twenty_fifth, median, seventy_fifth, maximum]

dataset_merged["price_label"] = pd.cut(x = dataset_merged["listing_price"], 
                                       labels = labels, 
                                       bins = bins)

# Only filtering for Adidas and Nike
adidas_and_nike = dataset_merged[(dataset_merged["brand"] == "Adidas") | (dataset_merged["brand"] == "Nike")]

# Grouping based on pricing labels
adidas_vs_nike = adidas_and_nike.groupby(["brand", "price_label"]).agg(num_products = ("product_id", "count"),
                                          mean_revenue = ("revenue", "mean")).round(2)
adidas_vs_nike

num_products  mean_revenue
brand  price_label                            
Adidas Budget                574       2015.68
       Average               655       3035.30
       Expensive             759       4621.56
       Elite                 587       8302.78
Nike   Budget                360       1654.40
       Average                 8        675.59
       Expensive              47        500.56
       Elite                 130       1367.45

In [111]:
# Answering the second question
# Merging the product description and rating datasets
dataset_merged_two = pd.merge(left = info,
                             right = reviews,
                             on = "product_id",
                             how = "left")
dataset_merged_two.dropna(inplace = True)

# Creating the "description_length" column
labels_two = ["0", "100", "200", "300", "400", "500", "600"]
bins_two = list(range(0, 800, 100))

dataset_merged_two["description_length"] = pd.cut(x = dataset_merged_two["description"].str.len(),
                                                  labels = labels_two,
                                                  bins = bins_two)

description_lengths = dataset_merged_two.groupby("description_length").agg(mean_rating = ("rating", "mean"),
                                                                          total_reviews = ("reviews", "sum")).round(2)
description_lengths

,mean_rating,total_reviews
description_length,,
0,2.26,36.0
100,3.19,17719.0
200,3.28,76115.0
300,3.29,28994.0
400,3.35,4984.0
500,3.12,852.0
600,3.65,818.0
